In [1]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split

In [2]:
import pickle
from sklearn.model_selection import train_test_split

In [3]:
train_data = pd.read_csv("C:/Users/13122/Desktop/rs/Stars/DataBank/quora-question-pairs/train.csv")
test_data = pd.read_csv("C:/Users/13122/Desktop/rs/Stars/DataBank/quora-question-pairs/test.csv")

In [4]:
train_data.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
mask = train_data.isnull().any(axis = 1)
train_data = train_data[~mask]
train_data

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [6]:
duplicate_data = train_data.query("is_duplicate == 1")
duplicate_data

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1
...,...,...,...,...,...,...
404280,404280,537922,537923,What are some outfit ideas to wear to a frat p...,What are some outfit ideas wear to a frat them...,1
404281,404281,99131,81495,Why is Manaphy childish in Pokémon Ranger and ...,Why is Manaphy annoying in Pokemon ranger and ...,1
404282,404282,1931,16773,How does a long distance relationship work?,How are long distance relationships maintained?,1
404284,404284,537926,537927,What does Jainism say about homosexuality?,What does Jainism say about Gays and Homosexua...,1


In [8]:
data_q = np.array(train_data.loc[:, ["question1","question2"]])
data_a = np.array(train_data.loc[:, "is_duplicate"])

In [20]:
def embed_sentences(sentences):
    # Load the Universal Sentence Encoder module
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
    model = hub.load(module_url)
    # Generate embeddings for the sentences
    embeddings = model(sentences)
    return embeddings

#data_q1 = data_q[: , 0]
#data_q2 = data_q[: , 1]
#q12_cca = np.concatenate((data_q1, data_q2), axis = 0)
#q12_cca_list = list(q12_cca)
#q12_embed = embed_sentences(q12_cca_list)

In [10]:
with open("C:/Users/13122/Desktop/rs/Stars/DataBank/quora-question-pairs/quora_emb.txt", 'rb') as f:
    q12_embed = pickle.load(f)

In [13]:
q12_embed_arr = np.array(q12_embed)

In [14]:
data_q1_embed = q12_embed_arr[0 : int(q12_embed_arr.shape[0]/2)]
data_q2_embed = q12_embed_arr[int(q12_embed_arr.shape[0]/2) : ]

In [15]:
data_q12_embed = np.concatenate((data_q1_embed, data_q2_embed), axis = 1)
data_a = data_a.reshape(404287,1)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(data_q12_embed, data_a, test_size = 0.20, random_state = 34)

In [28]:
def NNmodel():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512,  input_shape = (1024,), activation = "relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256, activation = "relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(16, activation = "relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ]) 
    return model

modelNN = NNmodel()
modelNN.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss= "binary_crossentropy", metrics = ["acc"])
modelNN.fit(x_train, y_train,epochs=12, batch_size =1024, validation_data=(x_test, y_test))

Epoch 1/12
316/316 [==============================] - 9s 28ms/step - loss: 0.4675 - acc: 0.7678 - val_loss: 0.3887 - val_acc: 0.8159
Epoch 2/12
316/316 [==============================] - 9s 27ms/step - loss: 0.3709 - acc: 0.8249 - val_loss: 0.3622 - val_acc: 0.8303
Epoch 3/12
316/316 [==============================] - 8s 27ms/step - loss: 0.3370 - acc: 0.8454 - val_loss: 0.3523 - val_acc: 0.8369
Epoch 4/12
316/316 [==============================] - 9s 28ms/step - loss: 0.3132 - acc: 0.8577 - val_loss: 0.3478 - val_acc: 0.8389
Epoch 5/12
316/316 [==============================] - 8s 27ms/step - loss: 0.2938 - acc: 0.8690 - val_loss: 0.3454 - val_acc: 0.8406
Epoch 6/12
316/316 [==============================] - 9s 29ms/step - loss: 0.2764 - acc: 0.8781 - val_loss: 0.3475 - val_acc: 0.8440
Epoch 7/12
316/316 [==============================] - 11s 36ms/step - loss: 0.2615 - acc: 0.8857 - val_loss: 0.3478 - val_acc: 0.8435
Epoch 8/12
316/316 [==============================] - 38s 119ms/step

In [29]:
modelNN.fit(x_train, y_train,epochs=6, batch_size =1024, validation_data=(x_test, y_test))

Epoch 1/6
316/316 [==============================] - 9s 27ms/step - loss: 0.1986 - acc: 0.9169 - val_loss: 0.3646 - val_acc: 0.8484
Epoch 2/6
316/316 [==============================] - 8s 27ms/step - loss: 0.1911 - acc: 0.9201 - val_loss: 0.3691 - val_acc: 0.8484
Epoch 3/6
316/316 [==============================] - 9s 29ms/step - loss: 0.1837 - acc: 0.9240 - val_loss: 0.3803 - val_acc: 0.8489
Epoch 4/6
316/316 [==============================] - 9s 29ms/step - loss: 0.1771 - acc: 0.9271 - val_loss: 0.3867 - val_acc: 0.8501
Epoch 5/6
316/316 [==============================] - 17s 53ms/step - loss: 0.1723 - acc: 0.9286 - val_loss: 0.3862 - val_acc: 0.8516
Epoch 6/6
316/316 [==============================] - 37s 116ms/step - loss: 0.1647 - acc: 0.9326 - val_loss: 0.3955 - val_acc: 0.8505


In [30]:
modelNN.evaluate(x_test, y_test)

2527/2527 [==============================] - 7s 3ms/step - loss: 0.3955 - acc: 0.8505


[0.3954944610595703, 0.8505033254623413]

In [69]:
with open("/Users/13122/Desktop/rs/Stars/DataBank/dataset_doc2vec.json", "r") as file:
    data = json.load(file)

def data_summary(data):
    num_sentences = data["overview"]["numParagraphs"]
    text_data = [data["data"][i]["text"] for i in range(num_sentences)] 
    query = data["query"]
    rank_high = data["overview"]["example_ranking"]["high"]
    rank_med = data["overview"]["example_ranking"]["medium"]
    rank_low = data["overview"]["example_ranking"]["low"]
    rank_non = data["overview"]["example_ranking"]["none"]
    return (num_sentences, query, text_data, rank_high, rank_med, rank_low, rank_non)

def one_hot_encoder(num_sentences ,rank_high, rank_med, rank_low, rank_non):
    one_hot = []
    index = [i+1 for i in range(num_sentences)]
    for i in index:
        if i in rank_high:
            one_hot.append([1])
        elif i in rank_med:
            one_hot.append([0])
        elif i in rank_low:
            one_hot.append([0])
        else:
            one_hot.append([0])
    return one_hot

def data_embed(query,text_data,mode = "normal"):
    if mode == "normal":
        Input = [query] + text_data
        emb_vec = embed_sentences(Input)
        sentence_embed = emb_vec[1: ]
        query_embed = emb_vec[0]
        return (query_embed, sentence_embed)
    
    elif mode == "split_average":
        text_data_split = [text_data[i].split(". ") for i in range(len(text_data))]
        text_data_split_exp = []
        for i in text_data_split:
            text_data_split_exp += i
        Input_split = [query] + text_data_split_exp
        emb_vec_split = embed_sentences(Input_split)
        query_embed_split = emb_vec_split[0]
        sentence_embed_split = emb_vec_split[1:]
        split_length = [len(i) for i in text_data_split]
        sentence_embed_split_ave = []
        t = 0
        for i in range(len(text_data_split)):
            split_ave = np.mean(sentence_embed_split[t: t + split_length[i]], axis = 0)
            sentence_embed_split_ave.append(split_ave)
            t += split_length[i]
        return (query_embed_split, sentence_embed_split_ave)
    else:
        return True

num_sentences,query, text_data, rank_high, rank_med, rank_low, rank_non = data_summary(data)
one_hot = np.array(one_hot_encoder(num_sentences, rank_high, rank_med, rank_low, rank_non))
query_embed, sentence_embed = data_embed(query, text_data, mode="split_average")
sentence_embed = np.array(sentence_embed)
query_embed = np.array(query_embed)
query_embed = query_embed.reshape([1,512])
query_embed_b = np.broadcast_to(query_embed, (sentence_embed.shape[0],query_embed.shape[1]))
sentence_embed_p = np.concatenate((query_embed_b, sentence_embed), axis = 1)
x_train_m, x_test_m, y_train_m, y_test_m = train_test_split(sentence_embed_p, one_hot, test_size = 0.20, random_state = 34)

In [44]:
modelNN.evaluate(x_train_m, y_train_m)


6/6 [==============================] - 0s 2ms/step - loss: 7.0128 - acc: 0.4489


[7.012835502624512, 0.4488636255264282]

In [45]:
modelNN.evaluate(x_test_m, y_test_m)

2/2 [==============================] - 0s 7ms/step - loss: 5.6604 - acc: 0.5227


[5.66044807434082, 0.5227272510528564]

In [34]:
modelNN.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 16)                4112      
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 660,257
Trainable params: 660,257
Non-tr

In [70]:
#transfer learning#
# Step 1: Create a new model with the same architecture as modelNN
new_model = NNmodel()

# Step 2: Load the weights from modelNN into the new model
new_model.set_weights(modelNN.get_weights())

# Step 3: Set all layers except the last one to trainable=False
for layer in new_model.layers[:-2]:
    layer.trainable = False
    
new_model.pop()
new_model.pop()
new_model.add(tf.keras.layers.Dense(64, activation="relu"))
new_model.add(tf.keras.layers.Dropout(0.4))
new_model.add(tf.keras.layers.Dense(16, activation="relu"))
new_model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

# Step 4: Compile the new model with the optimizer, loss, and metrics for your new task
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["acc"])

# Step 5: Train the new model on your new data, including the validation set
new_model.fit(x_train_m, y_train_m, epochs=200, validation_data=(x_test_m, y_test_m))


Epoch 1/200
6/6 [==============================] - 1s 49ms/step - loss: 0.7906 - acc: 0.3177 - val_loss: 0.6755 - val_acc: 0.6735
Epoch 2/200
6/6 [==============================] - 0s 14ms/step - loss: 0.6953 - acc: 0.5365 - val_loss: 0.6539 - val_acc: 0.6735
Epoch 3/200
6/6 [==============================] - 0s 11ms/step - loss: 0.6437 - acc: 0.7031 - val_loss: 0.6369 - val_acc: 0.6735
Epoch 4/200
6/6 [==============================] - 0s 15ms/step - loss: 0.6354 - acc: 0.7083 - val_loss: 0.6259 - val_acc: 0.6735
Epoch 5/200
6/6 [==============================] - 0s 14ms/step - loss: 0.6245 - acc: 0.7135 - val_loss: 0.6243 - val_acc: 0.6735
Epoch 6/200
6/6 [==============================] - 0s 17ms/step - loss: 0.6186 - acc: 0.7188 - val_loss: 0.6259 - val_acc: 0.6735
Epoch 7/200
6/6 [==============================] - 0s 14ms/step - loss: 0.6227 - acc: 0.7188 - val_loss: 0.6248 - val_acc: 0.6735
Epoch 8/200
6/6 [==============================] - 0s 18ms/step - loss: 0.6180 - acc: 0.71

6/6 [==============================] - 0s 13ms/step - loss: 0.5516 - acc: 0.7396 - val_loss: 0.5697 - val_acc: 0.6939
Epoch 64/200
6/6 [==============================] - 0s 14ms/step - loss: 0.5392 - acc: 0.7292 - val_loss: 0.5690 - val_acc: 0.6939
Epoch 65/200
6/6 [==============================] - 0s 12ms/step - loss: 0.5547 - acc: 0.7344 - val_loss: 0.5754 - val_acc: 0.6939
Epoch 66/200
6/6 [==============================] - 0s 13ms/step - loss: 0.5416 - acc: 0.7396 - val_loss: 0.5721 - val_acc: 0.6939
Epoch 67/200
6/6 [==============================] - 0s 13ms/step - loss: 0.5666 - acc: 0.7188 - val_loss: 0.5693 - val_acc: 0.6939
Epoch 68/200
6/6 [==============================] - 0s 14ms/step - loss: 0.5708 - acc: 0.7188 - val_loss: 0.5677 - val_acc: 0.6939
Epoch 69/200
6/6 [==============================] - 0s 13ms/step - loss: 0.5203 - acc: 0.7448 - val_loss: 0.5713 - val_acc: 0.6939
Epoch 70/200
6/6 [==============================] - 0s 13ms/step - loss: 0.5339 - acc: 0.7188 - 

6/6 [==============================] - 0s 13ms/step - loss: 0.5207 - acc: 0.7031 - val_loss: 0.5618 - val_acc: 0.7347
Epoch 126/200
6/6 [==============================] - 0s 14ms/step - loss: 0.5006 - acc: 0.7292 - val_loss: 0.5654 - val_acc: 0.7347
Epoch 127/200
6/6 [==============================] - 0s 14ms/step - loss: 0.4823 - acc: 0.7656 - val_loss: 0.5741 - val_acc: 0.7347
Epoch 128/200
6/6 [==============================] - 0s 13ms/step - loss: 0.5468 - acc: 0.7292 - val_loss: 0.5678 - val_acc: 0.7347
Epoch 129/200
6/6 [==============================] - 0s 11ms/step - loss: 0.4655 - acc: 0.7812 - val_loss: 0.5641 - val_acc: 0.7143
Epoch 130/200
6/6 [==============================] - 0s 15ms/step - loss: 0.4992 - acc: 0.7760 - val_loss: 0.5629 - val_acc: 0.7347
Epoch 131/200
6/6 [==============================] - 0s 17ms/step - loss: 0.5470 - acc: 0.7344 - val_loss: 0.5609 - val_acc: 0.7347
Epoch 132/200
6/6 [==============================] - 0s 14ms/step - loss: 0.5004 - acc: 0.

6/6 [==============================] - 0s 15ms/step - loss: 0.4672 - acc: 0.7760 - val_loss: 0.5579 - val_acc: 0.7347
Epoch 188/200
6/6 [==============================] - 0s 15ms/step - loss: 0.5218 - acc: 0.7188 - val_loss: 0.5482 - val_acc: 0.7551
Epoch 189/200
6/6 [==============================] - 0s 13ms/step - loss: 0.4844 - acc: 0.7188 - val_loss: 0.5498 - val_acc: 0.7347
Epoch 190/200
6/6 [==============================] - 0s 14ms/step - loss: 0.5086 - acc: 0.7760 - val_loss: 0.5513 - val_acc: 0.7347
Epoch 191/200
6/6 [==============================] - 0s 16ms/step - loss: 0.4745 - acc: 0.7760 - val_loss: 0.5501 - val_acc: 0.7551
Epoch 192/200
6/6 [==============================] - 0s 16ms/step - loss: 0.5235 - acc: 0.7083 - val_loss: 0.5490 - val_acc: 0.7551
Epoch 193/200
6/6 [==============================] - 0s 15ms/step - loss: 0.4788 - acc: 0.7760 - val_loss: 0.5516 - val_acc: 0.7347
Epoch 194/200
6/6 [==============================] - 0s 17ms/step - loss: 0.4788 - acc: 0.

In [71]:
new_model.fit(x_train_m, y_train_m, epochs=200, validation_data=(x_test_m, y_test_m))

Epoch 1/200
6/6 [==============================] - 0s 21ms/step - loss: 0.4868 - acc: 0.7760 - val_loss: 0.5627 - val_acc: 0.7143
Epoch 2/200
6/6 [==============================] - 0s 16ms/step - loss: 0.4902 - acc: 0.7500 - val_loss: 0.5720 - val_acc: 0.7143
Epoch 3/200
6/6 [==============================] - 0s 13ms/step - loss: 0.5327 - acc: 0.7292 - val_loss: 0.5706 - val_acc: 0.6939
Epoch 4/200
6/6 [==============================] - 0s 15ms/step - loss: 0.4820 - acc: 0.7292 - val_loss: 0.5683 - val_acc: 0.7143
Epoch 5/200
6/6 [==============================] - 0s 14ms/step - loss: 0.4800 - acc: 0.7969 - val_loss: 0.5646 - val_acc: 0.7143
Epoch 6/200
6/6 [==============================] - 0s 13ms/step - loss: 0.5201 - acc: 0.7240 - val_loss: 0.5667 - val_acc: 0.7551
Epoch 7/200
6/6 [==============================] - 0s 18ms/step - loss: 0.4929 - acc: 0.7656 - val_loss: 0.5727 - val_acc: 0.7143
Epoch 8/200
6/6 [==============================] - 0s 18ms/step - loss: 0.4882 - acc: 0.74

6/6 [==============================] - 0s 18ms/step - loss: 0.4896 - acc: 0.7552 - val_loss: 0.5802 - val_acc: 0.7551
Epoch 64/200
6/6 [==============================] - 0s 13ms/step - loss: 0.4987 - acc: 0.7812 - val_loss: 0.5779 - val_acc: 0.7143
Epoch 65/200
6/6 [==============================] - 0s 14ms/step - loss: 0.4764 - acc: 0.7708 - val_loss: 0.5697 - val_acc: 0.7347
Epoch 66/200
6/6 [==============================] - 0s 16ms/step - loss: 0.4843 - acc: 0.7760 - val_loss: 0.5746 - val_acc: 0.7143
Epoch 67/200
6/6 [==============================] - 0s 14ms/step - loss: 0.4731 - acc: 0.8073 - val_loss: 0.5722 - val_acc: 0.7347
Epoch 68/200
6/6 [==============================] - 0s 11ms/step - loss: 0.4941 - acc: 0.7604 - val_loss: 0.5733 - val_acc: 0.6939
Epoch 69/200
6/6 [==============================] - 0s 15ms/step - loss: 0.4570 - acc: 0.7656 - val_loss: 0.5807 - val_acc: 0.6939
Epoch 70/200
6/6 [==============================] - 0s 13ms/step - loss: 0.4858 - acc: 0.7604 - 

6/6 [==============================] - 0s 20ms/step - loss: 0.4437 - acc: 0.7969 - val_loss: 0.5856 - val_acc: 0.6939
Epoch 126/200
6/6 [==============================] - 0s 16ms/step - loss: 0.4906 - acc: 0.7344 - val_loss: 0.5794 - val_acc: 0.7347
Epoch 127/200
6/6 [==============================] - 0s 13ms/step - loss: 0.4740 - acc: 0.7604 - val_loss: 0.5841 - val_acc: 0.7143
Epoch 128/200
6/6 [==============================] - 0s 19ms/step - loss: 0.4400 - acc: 0.7865 - val_loss: 0.5828 - val_acc: 0.7143
Epoch 129/200
6/6 [==============================] - 0s 12ms/step - loss: 0.5208 - acc: 0.7344 - val_loss: 0.5865 - val_acc: 0.7347
Epoch 130/200
6/6 [==============================] - 0s 12ms/step - loss: 0.4270 - acc: 0.7812 - val_loss: 0.5873 - val_acc: 0.7347
Epoch 131/200
6/6 [==============================] - 0s 14ms/step - loss: 0.4571 - acc: 0.7552 - val_loss: 0.5787 - val_acc: 0.7347
Epoch 132/200
6/6 [==============================] - 0s 14ms/step - loss: 0.4851 - acc: 0.

6/6 [==============================] - 0s 16ms/step - loss: 0.4567 - acc: 0.7760 - val_loss: 0.5861 - val_acc: 0.7143
Epoch 188/200
6/6 [==============================] - 0s 14ms/step - loss: 0.4892 - acc: 0.7760 - val_loss: 0.5836 - val_acc: 0.7143
Epoch 189/200
6/6 [==============================] - 0s 11ms/step - loss: 0.4932 - acc: 0.8021 - val_loss: 0.5704 - val_acc: 0.7143
Epoch 190/200
6/6 [==============================] - 0s 12ms/step - loss: 0.5656 - acc: 0.7031 - val_loss: 0.5741 - val_acc: 0.6939
Epoch 191/200
6/6 [==============================] - 0s 14ms/step - loss: 0.4420 - acc: 0.7969 - val_loss: 0.5787 - val_acc: 0.6939
Epoch 192/200
6/6 [==============================] - 0s 13ms/step - loss: 0.4397 - acc: 0.7812 - val_loss: 0.5789 - val_acc: 0.6939
Epoch 193/200
6/6 [==============================] - 0s 17ms/step - loss: 0.4751 - acc: 0.7552 - val_loss: 0.5738 - val_acc: 0.6939
Epoch 194/200
6/6 [==============================] - 0s 16ms/step - loss: 0.4613 - acc: 0.

In [72]:
#to test moral, just change the file name to be dataset_moralStandards.json, and then run below code#


In [66]:
new_model.evaluate(sentence_embed_p, one_hot)

8/8 [==============================] - 0s 4ms/step - loss: 0.4131 - acc: 0.8091


[0.4130723178386688, 0.8091286420822144]

2/2 [==============================] - 0s 22ms/step - loss: 0.5636 - acc: 0.7551


[0.5635823011398315, 0.7551020383834839]